In [1]:
import os
import pandas as pd
import torch
from torch_geometric.loader import DataLoader
import mlflow
from datetime import datetime

from src.modules.graph_nn.dataset import TransactionDataset
from src.modules.graph_nn.model import TransactionGNN
from src.modules.graph_nn.train import train_epoch, evaluate_epoch


class Constants:
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BATCH_SIZE = 32
    HIDDEN_DIM = 256
    NUM_LAYERS = 5
    NUM_HEADS = 4
    SHUFFLE = False
    TRAIN_SPLIT = 0.8

    TRANSACTIONS_PATH = "/Users/alfa/Documents/diplom/graphnn-recommendation-system/data/processed_transactions_train.csv"

In [2]:
# Загружаем транзакции и создаем torch dataset
transactions = pd.read_parquet(Constants.TRANSACTIONS_PATH)
graph_dataset = TransactionDataset(transactions)

# Разделяем на train и test
graph_train_size = int(Constants.TRAIN_SPLIT * len(graph_dataset))
graph_train_dataset = torch.utils.data.Subset(graph_dataset, range(graph_train_size))
graph_test_dataset = torch.utils.data.Subset(
    graph_dataset, range(graph_train_size, len(graph_dataset))
)

# Создаем загрузчики данных
graph_train_loader = DataLoader(
    graph_train_dataset, batch_size=Constants.BATCH_SIZE, shuffle=Constants.SHUFFLE
)
graph_test_loader = DataLoader(
    graph_test_dataset, batch_size=Constants.BATCH_SIZE, shuffle=Constants.SHUFFLE
)

# Инициализируем модель
graph_model = TransactionGNN(
    num_items=graph_dataset.num_items,
    hidden_dim=Constants.HIDDEN_DIM,
    num_layers=Constants.NUM_LAYERS,
    num_heads=Constants.NUM_HEADS,
).to(Constants.DEVICE)
graph_optimizer = torch.optim.Adam(graph_model.parameters(), lr=0.001)

# Обучение и оценка
for epoch in range(100):
    train_loss = train_epoch(
        model=graph_model,
        loader=graph_train_loader,
        optimizer=graph_optimizer,
        device=Constants.DEVICE,
    )
    test_acc, test_loss = evaluate_epoch(
        model=graph_model,
        loader=graph_test_loader,
        device=Constants.DEVICE,
    )
    print(
        f"Epoch: {epoch:03d}, Train Loss: {train_loss:.4f}, Test Acc: {test_acc:.4f}, Test Loss: {test_loss:.4f}"
    )

Evaluate Epoch:: 100%|██████████| 35/35 [00:00<00:00, 280.63it/s]


Epoch: 000, Train Loss: 4.5858, Test Acc: 0.0275, Test Loss: 5.3499


Evaluate Epoch:: 100%|██████████| 35/35 [00:00<00:00, 264.11it/s]


Epoch: 001, Train Loss: 4.0653, Test Acc: 0.0229, Test Loss: 4.6426


Evaluate Epoch:: 100%|██████████| 35/35 [00:00<00:00, 287.06it/s]


...

Epoch: 097, Train Loss: 2.5111, Test Acc: 0.1366, Test Loss: 6.7178


Evaluate Epoch:: 100%|██████████| 35/35 [00:00<00:00, 258.49it/s]


Epoch: 098, Train Loss: 2.4396, Test Acc: 0.1489, Test Loss: 7.6190


Evaluate Epoch:: 100%|██████████| 35/35 [00:00<00:00, 257.25it/s]

Epoch: 099, Train Loss: 2.5495, Test Acc: 0.1494, Test Loss: 6.6917


# Логирование артефактов моделей в MLFlow

In [3]:
os.environ["AWS_ACCESS_KEY_ID"] = "minio"
os.environ["AWS_SECRET_ACCESS_KEY"] = "minio123"
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"

In [4]:
def save_model_to_mlflow():
    mlflow.set_tracking_uri("http://localhost:5000")
    mlflow.set_experiment("graph-nn-model")

    with mlflow.start_run(run_name=f"model_{datetime.now().strftime('%Y%m%d_%H%M%S')}"):
        # Сохраняем модель PyTorch
        mlflow.pytorch.log_model(graph_model, "model")

        # Сохраняем энкодер товаров
        import joblib

        joblib.dump(graph_dataset.item_encoder, "item_encoder.pkl")
        mlflow.log_artifact("item_encoder.pkl", "encoders")

        # Логируем параметры модели
        mlflow.log_params(
            {
                "num_items": graph_dataset.num_items,
                "hidden_dim": Constants.HIDDEN_DIM,
                "num_layers": Constants.NUM_LAYERS,
                "num_heads": Constants.NUM_HEADS,
            }
        )


save_model_to_mlflow()

2025/04/16 21:25:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run model_20250416_212504 at: http://localhost:5000/#/experiments/3/runs/43d2dbcfc2e2476aa6d996dcf276ec4c
🧪 View experiment at: http://localhost:5000/#/experiments/3
